In [1]:
## dependencies

In [4]:
#pip install country_converter --upgrade

import hvplot.pandas
import pandas as pd
import requests

In [6]:
#Load the CSV file created in Part 1 into a Pandas DataFrame
hotel_df = pd.read_csv("hotel_bookings.csv")
country_df = pd.read_csv("countries_codes_and_coordinates.csv")

In [7]:
country_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Country              262 non-null    object
 1   Alpha-2 code         262 non-null    object
 2   Alpha-3 code         262 non-null    object
 3   Numeric code         262 non-null    object
 4   Latitude (average)   262 non-null    object
 5   Longitude (average)  262 non-null    object
dtypes: object(6)
memory usage: 12.4+ KB


In [ ]:
hotels_df.desc?